In [8]:
import pandas as pd
import requests
import urllib.parse
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
# LISTAR CANDIDATOS

#candidatos = pd.read_csv('gobernadores_regionales.csv')
candidatos = pd.read_csv('vicegobernadores_regionales.csv')

dict = {}
for dni_candidato in candidatos['txDocumentoIdentidad']:
    try:
        url = "https://infogob.jne.gob.pe/Politico/ListarPolitico"
        headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"}
        body = {"IdDNI": dni_candidato, "token":"cA6%2B6LtErj8=6L"}
        resp = requests.post(url=url, headers=headers, data=body, verify=False)
        data = resp.json()["Data"][0]["TxRutaPolitico"].split("partidario_",1)[1]
        data_encoded = urllib.parse.quote(data.encode('utf8'))
        dict[dni_candidato] = data_encoded
        print(data_encoded)
    except:
        continue
df = pd.DataFrame(dict.items(), columns=["DNI","URL"])
#df.to_csv("dnis_urls_gobernadores_regionales.csv")
df.to_csv("dnis_urls_vicegobernadores_regionales.csv")

In [15]:
# HISTORIAL PARTIDARIO

from requests_html import HTMLSession
session = HTMLSession()

#candidatos = pd.read_csv('dnis_urls_gobernadores_regionales.csv', index_col=0)
candidatos = pd.read_csv('dnis_urls_vicegobernadores_regionales.csv', index_col=0)

# DUMMY
# candidatos = pd.DataFrame({'DNI': [23821919, 22477150,22508609],
#                    'URL': ["Cx8y5zTyGCE%3D8z", "NsJFh4Uc780c6%2B%400ElOxMA%3D%3DJ4", "aT2peYCtZXE%3D2Y"]})

dict = {}
for dni_candidato, url_candidato in candidatos.itertuples(index=False):
    dict[dni_candidato] = {}
    try:
        body = {"token": "KOlSDC8AUxc=lC"}
        r = session.post(f"https://infogob.jne.gob.pe/Politico/_HistorialFichaPolitico?istrParameters={url_candidato}", verify=False, data=body)

        about = r.html.find('.alias')
        for idx, partido in enumerate(about):
            dict[dni_candidato][idx] = partido.text

    except:
        continue

df = pd.DataFrame.from_dict(dict, orient="index")
#df.to_csv("historial_partidario_gobernadores_regionales.csv")
df.to_csv("historial_partidario_vicegobernadores_regionales.csv")